# Ingress Services

### Introduction

Now so far, we have seen that it's possible to expose our services externally by choosing a service of type `NodePort` instead of a `ClusterIP` service.  But really using the `NodePort` service is not a best practice.  Instead, we'll use an ingress service which will then route requests to any of other kubernetes services.  As we'll the ingress service works similarly to routing in flask.  It allows for external access, and also forwards requests to one of multiple services based on the requested path.  

We'll see how this works in this lesson.

### Ingress Services

Now sitting in front of our services, are generally two other components -- the load balancer and the ingress service.  The load balancer is designed to be highly available, and generally does not provide a routing function, so it is generally the first thing a request goes to when reaching our application.  

From there, the load balancer forwards the request to the ingress service.  

As we can see in the diagram below, this ingress service performs the task of routing requests to different services. This routing task happens based on the request path.  So for example, if our domain name is `apartments.com`, we can decide to take a request to the backend service if the request is to `apartments.com/api`, and to the frontend service if the path is just to `apartments.com`.

<img src="./multilayer.jpg" width="60%">

Now to simplify things for this lesson, we'll just focus on setting up our ingress service.  And we'll only have our ingress service forward requests to our frontend service.

<img src="./with-ingress.jpg" width="40%">

### Installing the Ingress

Now the ingress service we'll be using, `Nginx ingress`, is not included with kubernetes out of the box.  Instead we'll need to include it as an `addon`.

We can see a list of addons available from minikube if we run:

`minikube addons list`

<img src="./addons_list.png" width="60%">

There, we'll see an addon for `ingress`.  So to install the service we run:

`minikube addons enable ingress`

> And then we can confirm that the service is starting by viewing the pods under kube-system, and looking for the ingress-nginx pods.

<img src="./enable_ingress.png" width="80%">

> Notice that above we call get pods with the `kube-system` namespace.

Now we can see that minikube has set things up such that nginx is now running on the ip address.  We can see this by visiting our minikube ip.

<img src="./nginx.png" width="60%">

### Building the Ingress

Let's just see show the ingress config file.  Read through it and see what you can understand -- we'll explain it below.

```yaml
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
    name: public-routing
spec:
    rules:
        - host: apartments.com
          http:
            paths:
                - path: /
                  pathType: Prefix
                  backend: 
                    service:
                      name: frontend-service
                      port: 
                        number: 8501
```

So the first few keys are fairly typical.  Here, we are using an object that we load up from the `networking.k8s.io/v1` module, and specify a type of `Ingress`.  Then we give our object a name, just like we did for the rest of our objects.

Now the `spec` sets up a series of routes just like we would see in a flask controller.  Under the key of `rules`, we first specify the `host` key, as Kubernetes anticipates that we may want different rules for different domain names.  

> Here, we'll say that our domain name is `apartments.com`.  

The `backend` key which we see under path, allows us to be taken to a service.  So we will effectively always use the key of `backend` under the path.  

Then we specify the service the request is forwarded to when reaching the specified path.  So here, the request is forwarded to the `frontend-service`, port `8501`, which is where our streamlit app is exposed.  

Finally, notice that we specified the `pathType` of `Prefix`.  There are two options -- either `Prefix` or `Exact`.  With the Prefix specification, kubernetes will ensure that anything up to the next `/` is a match.  So if our path is `/foo` and the request is `/foo/bar`, this is match.  

### Setting up a domain name

Now we don't actually have an `apartments.com` domain name.  But we can trick our computer into thinking we do, by changing our `/etc/hosts` file in linux on our host computer.

`sudo vim /etc/hosts` 

And below, we added a domain name with an ip address that matches our minikube ip.  
```bash
##
# Host Database
#
# localhost is used to configure the loopback interface
# when the system is booting.  Do not change this entry.
##
127.0.0.1	localhost
255.255.255.255	broadcasthost
::1             localhost
# Added by Docker Desktop
# To allow the same kube context to work on the host and the container:
127.0.0.1 kubernetes.docker.internal
# End of section
192.168.64.9 apartments.com
```

So now from our local computer, if we visit `apartments.com`, our computer will go to our minikube ip.

<img src="./apartments_nginx.png" width="60%">

And now we can simply apply our ingress service to have kubernetes forward our request along to our `frontend-service` where our streamlit app is hosted.

`kubectl apply -f ingress.yaml`

And then confirm that our ingress was created with a call to `describe ingress`.

<img src="./describe-ingress.png" width='100%'>

And from there, we can see our application at the specified domain.

<img src="./apartments_streamlit.png" width="50%">

So we are able to access our website at apartments.com because we have the domain pointing to the ingress service, and the ingress service specifies to connect requests to `/`, the root path, to the `frontend-service`.

### Summary

In this lesson, we learned how to expose our cluster externally using an ingress service. Specifically, we used the [ingress nginx service](https://github.com/kubernetes/ingress-nginx/).  We first installed the service locally to kubernetes, and then wrote our configuration file.  

Our file specifies to rout a request to `/` to our `frontend-service` on port 8501.

### Resources

[Kubernetes Nginx](https://kubernetes.io/docs/concepts/services-networking/ingress/)

[Ingress Nginx Github](https://github.com/kubernetes/ingress-nginx)

[Kubernetes Ingress Blogpost](https://www.joyfulbikeshedding.com/blog/2018-03-26-studying-the-kubernetes-ingress-system.html)